<a href="https://colab.research.google.com/github/AlexKitano/Portfolio/blob/main/Portfolio_Classificacao_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos realizar uma análise nos dados para previsão de doenças cardíacas.

Fazendo a leitura de dados.

Fazer a leitura da base de dados, a partir do arquivo raw_merged_heart_dataset.csv'e utilizando o read_csv() da biblioteca pandas.

In [ ]:
import pandas as pd

In [ ]:
dados = pd.read_csv('/content/cleaned_merged_heart_dataset.csv')

In [ ]:
dados

Verificar informações adicionais dos dados, como a presença de dados nulos e tipos de dados em cada coluna, usando o método info().

In [ ]:
dados.info()

In [ ]:
dados["sex"].value_counts()

In [ ]:
dados["cp"].value_counts()

In [ ]:
dados["fbs"].value_counts()

In [ ]:
dados["restecg"].value_counts()

In [ ]:
dados["exang"].value_counts()

In [ ]:
dados["slope"].value_counts()

In [ ]:
dados["ca"].value_counts()

In [ ]:
dados["thal"].value_counts()

In [ ]:
dados["target"].value_counts()

In [ ]:
# Criar colunas codificadas:
categorical_columns = [col for col in ['cp', 'restecg', 'slope', 'ca', 'thal'] if col in dados.columns]

dados = pd.get_dummies(dados, columns=categorical_columns, drop_first=False)

dados

In [ ]:
dados = dados.replace({True: 1, False: 0})
dados

In [ ]:
# Colocar a coluna target como última coluna
target_column = dados.pop('target')
dados['target'] = target_column

dados

Análise Descritiva

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(8, 6))

sns.set_style("whitegrid")

ax = sns.boxplot(x = "target", y = "age", data=dados,
                 hue = "target", linewidth = 5, palette = "Set3")

ax.set_xlabel("Risco de Ataque Cardíaco e Derrame")
ax.set_ylabel("Idade")

handles, _ = ax.get_legend_handles_labels()
ax.legend(handles, ["0: Menos chance", "1: Mais chance"], fontsize = 12, bbox_to_anchor = (1.05, 1))
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))

sns.set_style("whitegrid")

ax = sns.boxplot(x = "target", y = "trestbps", data=dados,
                 hue = "target", linewidth = 5, palette = "Set3")

ax.set_xlabel("Risco de Ataque Cardíaco e Derrame")
ax.set_ylabel("Pressão Arterial em Repouso")

handles, _ = ax.get_legend_handles_labels()
ax.legend(handles, ["0: Menos chance", "1: Mais chance"], fontsize = 12, bbox_to_anchor = (1.05, 1))
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))

sns.set_style("whitegrid")

ax = sns.boxplot(x = "target", y = "chol", data=dados,
                 hue = "target", linewidth = 5, palette = "Set3")

ax.set_xlabel("Risco de Ataque Cardíaco e Derrame")
ax.set_ylabel("Nível de Colesterol Sérico")

handles, _ = ax.get_legend_handles_labels()
ax.legend(handles, ["0: Menos chance", "1: Mais chance"], fontsize = 12, bbox_to_anchor = (1.05, 1))
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))

sns.set_style("whitegrid")

ax = sns.boxplot(x = "target", y = "thalachh", data=dados,
                 hue = "target", linewidth = 5, palette = "Set3")

ax.set_xlabel("Risco de Ataque Cardíaco e Derrame")
ax.set_ylabel("Frequência Cardíaca Máxima Atingida")

handles, _ = ax.get_legend_handles_labels()
ax.legend(handles, ["0: Menos chance", "1: Mais chance"], fontsize = 12, bbox_to_anchor = (1.05, 1))
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))

sns.set_style("whitegrid")

ax = sns.boxplot(x = "target", y = "oldpeak", data=dados,
                 hue = "target", linewidth = 5, palette = "Set3")

ax.set_xlabel("Risco de Ataque Cardíaco e Derrame")
ax.set_ylabel("Depressão Do Segmento ST Induzida Pelo Exercício Em Relação Ao Repouso")

handles, _ = ax.get_legend_handles_labels()
ax.legend(handles, ["0: Menos chance", "1: Mais chance"], fontsize = 12, bbox_to_anchor = (1.05, 1))
plt.show()

Separar os dados em treino e teste

In [ ]:
X, y = dados.iloc[:, :-1], dados.iloc[:, -1]

In [ ]:
from sklearn.model_selection import train_test_split
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(random_state=0)

dt.fit(X_treino, y_treino)

preds = dt.predict(X_teste)

from sklearn.metrics import accuracy_score

acuracia = accuracy_score(y_teste, preds)
print(f'Acurácia: {acuracia:.2f}')

In [ ]:
from sklearn.metrics import classification_report

report = (classification_report(y_teste, preds))
print(report)

In [ ]:
from sklearn.metrics import RocCurveDisplay
RocCurveDisplay.from_predictions(y_teste, preds, name='Decision Tree');

Classificação com XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
xg_cl = xgb.XGBClassifier(objective='binary:logistic', enable_categorical=True)
xg_cl.fit(X_treino, y_treino)
preds = xg_cl.predict(X_teste)

acuracia = accuracy_score(y_teste, preds)
print(f'Acurácia: {acuracia:.2f}')

Analisando as métricas do XGBoost

In [ ]:
report = (classification_report(y_teste, preds))
print(report)

In [ ]:
RocCurveDisplay.from_predictions(y_teste, preds, name='XGBoost');

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(xg_cl, X_teste, y_teste, display_labels=['Ausencia', 'Presenca'],
                                      normalize='true', cmap='Blues');

Aplicando a validação cruzada

In [ ]:
dmatrix = xgb.DMatrix(data=X, label=y, enable_categorical=True)

In [ ]:
params = {"objective":"binary:logistic"}

cv_resultados = xgb.cv(dtrain=dmatrix, params=params, nfold=3, num_boost_round=5, metrics="error", as_pandas=True, seed=123)
print(cv_resultados)
acuracia = 1 - cv_resultados["test-error-mean"].iloc[-1]
print(f'Acurácia: {acuracia:.2f}')

In [ ]:
params = {"objective":"binary:logistic"}

cv_resultados = xgb.cv(dtrain=dmatrix, params=params, nfold=3, num_boost_round=50, metrics="error", as_pandas=True, seed=123)
print(cv_resultados)
acuracia = 1 - cv_resultados["test-error-mean"].iloc[-1]
print(f'Acurácia: {acuracia:.2f}')

Ajustando hiperparâmetros do XGboost

In [ ]:
params = {"objective":"binary:logistic"}

cv_resultados = xgb.cv(dtrain=dmatrix, params=params, nfold=3, num_boost_round=100,
                       early_stopping_rounds=3, metrics="error", as_pandas=True, seed=123)
print(cv_resultados)
acuracia = 1 - cv_resultados["test-error-mean"].iloc[-1]
print(f'Acurácia: {acuracia:.2f}')

In [ ]:
params = {"objective":"binary:logistic", "colsample_bytreee": 0.3,
          "learning_rate": 0.1, "max_depth": 5}

cv_resultados = xgb.cv(dtrain=dmatrix, params=params, nfold=3, num_boost_round=100,
                       early_stopping_rounds=3, metrics="error", as_pandas=True, seed=123)
print(cv_resultados)
acuracia = 1 - cv_resultados["test-error-mean"].iloc[-1]
print(f'Acurácia: {acuracia:.2f}')

Aplicando o Random Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    'max_depth': [0, 2, 4, 6, 10],
    'learning_rate': [0.1, 0.2, 0.3],
    'colsample_bytree': [0.1, 0.5, 1],
    'n_estimators': [10, 25, 50]
}

modelo_xgb = xgb.XGBClassifier()

random_search_cv = RandomizedSearchCV(estimator=modelo_xgb, param_distributions=params,
                                      n_iter=5, cv=4, scoring='accuracy', verbose=1)

random_search_cv.fit(X, y)

print('Melhores parametros encontrados: ', random_search_cv.best_params_)
print('Acuracia: ', random_search_cv.best_score_)

Aplicando o Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth': [0, 2, 4, 6, 10],
    'learning_rate': [0.1, 0.2, 0.3],
    'colsample_bytree': [0.1, 0.5, 1],
    'n_estimators': [10, 25, 50]
}

modelo_xgb = xgb.XGBClassifier()

grid_search_cv = GridSearchCV(estimator=modelo_xgb, param_grid=params, cv=4, scoring='accuracy', verbose=1)

grid_search_cv.fit(X, y)

print('Melhores parametros encontrados: ', grid_search_cv.best_params_)
print('Acuracia: ', grid_search_cv.best_score_)

In [ ]:
xg_cl_ajustado = xgb.XGBClassifier(colsample_bytree=1, learning_rate=0.3, max_depth=0, n_estimators=25)

xg_cl_ajustado.fit(X_treino, y_treino)

preds = xg_cl_ajustado.predict(X_teste)

acuracia = accuracy_score(y_teste, preds)
print(f'Acurácia: {acuracia:.2f}')

In [ ]:
report = (classification_report(y_teste, preds))
print(report)

In [ ]:
RocCurveDisplay.from_predictions(y_teste, preds, name='XGBoost ajustado');

In [ ]:
ConfusionMatrixDisplay.from_estimator(xg_cl_ajustado, X_teste, y_teste, display_labels=('Ausencia', 'Presenca'),
                                      normalize='true', cmap='Blues');

Incorporando XGBoost em pipelines

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('xgboost', xgb.XGBClassifier())])

xgb_param_grid = {
    'xgboost__max_depth': [0, 2, 4, 6, 10],
    'xgboost__learning_rate': [0.1, 0.2, 0.3],
    'xgboost__colsample_bytree': [0.1, 0.5, 1],
    'xgboost__n_estimators': [10, 25, 50]
}

xgb_grid_search = GridSearchCV(estimator=pipeline, param_grid=xgb_param_grid, cv=4, scoring='accuracy', verbose=1)

grid_search_cv.fit(X, y)

print('Acuracia: ', grid_search_cv.best_score_)
print('Melhor estimador:', grid_search_cv.best_estimator_)



Salvar o modelo

In [ ]:
import joblib

joblib.dump(grid_search_cv, 'modelo_pipeline.pkl')

Utilizando o modelo com dados novos

In [ ]:
modelo = joblib.load('modelo_pipeline.pkl')

In [ ]:
novos_pacientes = pd.read_excel('/content/novos_pacientes.xlsx')

In [ ]:
novos_pacientes

In [ ]:
categorical_columns = ['cp', 'restecg', 'slope', 'ca', 'thal']

# Aplicando get_dummies para novos pacientes
novos_pacientes = pd.get_dummies(novos_pacientes, columns=categorical_columns, drop_first=False)

# Garantir que todas as colunas dos dados de treinamento estejam presentes em novos_pacientesfor col in X_treino.columns:
    if col not in novos_pacientes.columns:
        novos_pacientes[col] = 0  # Add missing columns with 0 values

# Reordenar as colunas para corresponder à ordem dos dados de treinamentonovos_pacientes = novos_pacientes[X_treino.columns]

previsoes = modelo.predict(novos_pacientes)

In [ ]:
previsoes

In [ ]:
probabilidades = modelo.predict_proba(novos_pacientes)

In [ ]:
probabilidades

In [ ]:
df_novos_pacientes = pd.DataFrame(novos_pacientes)

df_novos_pacientes['previsao'] = previsoes

df_novos_pacientes